In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/movie_info.csv')
df = df.rename(columns={"locale_id": "local_id"})
df.head()

,local_id,title,id_imdb,genre_imdb
0,1,Toy Story (1995),tt0114709,Animation|Adventure|Comedy|Family|Fantasy
1,2,GoldenEye (1995),tt0113189,Action|Adventure|Thriller
2,3,Four Rooms (1995),tt0113101,Comedy
3,4,Get Shorty (1995),tt0113161,Comedy|Crime|Thriller
4,5,Copycat (1995),tt0112722,Drama|Mystery|Thriller


In [3]:
synopsis_data = pd.read_json('data/movie_synopsis.json')
synopsis_data = synopsis_data.transpose()
synopsis_data.head()

,imdbID,plot_synopsis,local_id,plot_source
1,tt0114709,A boy called Andy Davis (voice: John Morris) u...,1,imdb
2,tt0113189,"The story opens in 1986, in the Cold War Sovie...",2,imdb
3,tt0113101,The film begins with Ted the Bellhop (Tim Roth...,3,imdb
4,tt0113161,Chilli Palmer (John Travolta) is a loan shark ...,4,imdb
5,tt0112722,After giving a guest lecture on criminal psych...,5,imdb


In [4]:
movie_df = pd.merge(df, synopsis_data, on='local_id')
movie_df = movie_df.drop(['id_imdb'], axis=1)
movie_df.head()

,local_id,title,genre_imdb,imdbID,plot_synopsis,plot_source
0,1,Toy Story (1995),Animation|Adventure|Comedy|Family|Fantasy,tt0114709,A boy called Andy Davis (voice: John Morris) u...,imdb
1,2,GoldenEye (1995),Action|Adventure|Thriller,tt0113189,"The story opens in 1986, in the Cold War Sovie...",imdb
2,3,Four Rooms (1995),Comedy,tt0113101,The film begins with Ted the Bellhop (Tim Roth...,imdb
3,4,Get Shorty (1995),Comedy|Crime|Thriller,tt0113161,Chilli Palmer (John Travolta) is a loan shark ...,imdb
4,5,Copycat (1995),Drama|Mystery|Thriller,tt0112722,After giving a guest lecture on criminal psych...,imdb
